In [1]:
import numpy as np

import json
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from keras.optimizers import *

Using TensorFlow backend.


In [2]:
with open('data/sentipolc_word_index.json', 'r') as f:
    word_index = json.load(f)

In [3]:
with open('data/words.tsv', 'w') as f:
    for w,_ in word_index.items():
        f.write('{}\n'.format(w))

In [4]:
words = [w for w,_ in word_index.items()]

In [5]:
data = np.load('data/sentipolc_seq.npz')

x_train = data['x_train']
y_train = data['y_train']

x_test = data['x_test']
y_test = data['y_test']

Check the max length of the text

In [6]:
max_len_seq = max([len(x) for x in x_train])
print('max len seq {}'.format(max_len_seq))
max_idx = max(x_train.max())
print('max id {}'.format(max_idx))

max len seq 40
max id 15204


In [7]:
x_train_pad = sequence.pad_sequences(x_train, maxlen=max_len_seq, padding='post')
x_test_pad = sequence.pad_sequences(x_test, maxlen=max_len_seq, padding='post')

In [8]:
x_train_pad.shape

(7410, 40)

In [9]:
x_train_pad[:2]

array([[15204,    11,   980,    15, 15204, 15204,    22, 15204, 15204,
          109,    10, 15204,    34,    33, 15204, 15204, 15204,    47,
        15204,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0],
       [15204, 15204,  1381,   474, 15204, 15204, 15204,    47, 15204,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0]], dtype=int32)

In [10]:
from sklearn.model_selection import StratifiedShuffleSplit

split_train_test = StratifiedShuffleSplit(1,test_size=0.2, random_state=123456)

for train, test in split_train_test.split(x_train_pad, y_train):
    x_train_pad_split, y_train_pad_split = x_train_pad[train], y_train[train]
    x_val_pad_split, y_val_pad_split = x_train_pad[test], y_train[test]

In [11]:
callbacks = [
    TensorBoard(histogram_freq=1, batch_size=128, embeddings_freq=1),
    ReduceLROnPlateau(patience=2, verbose=1)
]

Instructions for updating:
Use the retry module or similar alternatives.


In [12]:
def build_model():
    model = Sequential()
    model.add(Embedding(input_dim=max_idx, output_dim=100, input_length=max_len_seq))
    model.add(Conv1D(filters=64, kernel_size=5, padding='same', activation='relu', strides=1))
    model.add(MaxPooling1D(2))
    model.add(Conv1D(filters=32, kernel_size=5, padding='same', activation='relu', strides=1))
    model.add(GlobalMaxPool1D())
    model.add(Dense(50, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
    
    return model

In [13]:
model = build_model()
model.summary()

Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 40, 100)           1520400   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 40, 64)            32064     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 20, 64)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 20, 32)            10272     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                1650      
___________________________________________________________

In [14]:
history = model.fit(x_train_pad_split, y_train_pad_split, validation_data=(x_val_pad_split, y_val_pad_split),
                    batch_size=128, validation_split=0.2, epochs=10, callbacks=callbacks)

Train on 5928 samples, validate on 1482 samples
Epoch 1/10
5928/5928 [==============================] - 1s 234us/step - loss: 0.6332 - acc: 0.6861 - val_loss: 0.6208 - val_acc: 0.6883
Epoch 2/10
5928/5928 [==============================] - 0s 55us/step - loss: 0.6161 - acc: 0.6883 - val_loss: 0.6116 - val_acc: 0.6883
Epoch 3/10
5928/5928 [==============================] - 0s 52us/step - loss: 0.5446 - acc: 0.6994 - val_loss: 0.6278 - val_acc: 0.7031
Epoch 4/10
5928/5928 [==============================] - 0s 52us/step - loss: 0.3619 - acc: 0.8440 - val_loss: 0.7597 - val_acc: 0.6734
Epoch 5/10
5928/5928 [==============================] - 0s 52us/step - loss: 0.2182 - acc: 0.9091 - val_loss: 0.9488 - val_acc: 0.6579

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 6/10
5928/5928 [==============================] - 0s 51us/step - loss: 0.1302 - acc: 0.9534 - val_loss: 0.9873 - val_acc: 0.6761
Epoch 7/10
5928/5928 [==============================] - 0s 

## Evaluate on the test set

In [15]:
callbacks = [
    TensorBoard(histogram_freq=0, batch_size=128, embeddings_freq=1),
]

In [16]:
model = build_model()

history = model.fit(x_train_pad, y_train, batch_size=128, epochs=3, callbacks=callbacks)

Epoch 1/3
7410/7410 [==============================] - 1s 88us/step - loss: 0.6367 - acc: 0.6789
Epoch 2/3
7410/7410 [==============================] - 0s 48us/step - loss: 0.6040 - acc: 0.6891
Epoch 3/3
7410/7410 [==============================] - 0s 48us/step - loss: 0.4879 - acc: 0.7615


In [17]:
model.evaluate(x_test_pad, y_test)

2000/2000 [==============================] - 0s 75us/step


[0.5743185014724731, 0.72350000000000003]

In [18]:
model.save('models/sentipolc_word_cnn.hdf5')